In [2]:
:opt no-lint

# Applicative Functor

## Outline

1. s

## Why `Applicative` functors?

### Our journey until now

The reason we need Applicative Functors (or Applicatives for short) is due to a limitation of regular `Functors`.

To provide a bit of context lets do a quick recap of our abstraction journey of `map` and `Functor`.

We started with a bunch of recursive functions:

```haskell
lowerString :: [Char] -> [Char]
lowerString []     = []
lowerString (x:xs) = toLower x : lowerString xs

addOne :: Num a => [a] -> [a]
addOne []     = []
addOne (x:xs) = (x + 1) : addOne xs

boolToBit :: [Bool] -> [Char]
boolToBit []     = []
boolToBit (x:xs) = (if x then '1' else '0') : boolToBit xs
```

These functions were usefull, but they were limited to applying a specific funtion to a list of specific types. However, we noticed that they had common traits that we could extract and we got the `map` function:

```haskell
map :: (a -> b) -> [a] -> [b]
map _ []     = []
map f (x:xs) = f x : map f xs


lowerString = map toLower

addOne = map (+1)

boolToBit = map (\x -> if x then '1' else '0')
```

The `map` function is a more powerful version of these functions because it's more general. Now, we can apply any function to a list of values. And because its more general, we can recreate the original functions by passing the concrete function we want to apply to the values.

But then, we faced a problem. `map` only works for lists. But there are plenty of structures with values we want to manipulate. And if we implement their own map-like functions:

```haskell
map :: (a -> b) -> [a] -> [b]
map _ []     = []
map f (x:xs) = f x : map f xs



maybeMap :: (a -> b) -> Maybe a -> Maybe b
maybeMap _ Nothing  = Nothing
maybeMap f (Just x) = Just (f x)



treeMap :: (a -> b) -> Tree a -> Tree b
treeMap f (Leaf x)       = Leaf (f x)
treeMap f (Node lt x rt) = Node (treeMap f lt) (f x) (treeMap f rt)
```

We realize that these functions were useful, but they were limited to specific structures. However, they had common traits that we could extract. And that's how we got the `Functor` type class:

A `Functor` is a type that can apply a function to the values of its structure without modifying the structure itself.

```haskell
class Functor f where
    fmap :: (a -> b) -> f a -> f b
    
    (<$) ::     a    -> f b -> f a
    (<$) =  fmap . const
    {-# MINIMAL fmap #-}
```
**Identity law**
```haskell
fmap id == id
```
**Composition law**
```haskell
fmap (f . g) == fmap f . fmap g
```

The `Functor` type class is a more powerful version of those functions because it's more general. Now, we can apply any function to any structure that is an instance of `Functor`. And, of course, we obtain what we had before, and more, by creating the instances: 

```haskell
instance Functor [] where
  -- fmap :: (a -> b) -> [a] -> [b]
  fmap _ []     = []
  fmap f (x:xs) = f x : fmap f xs


instance Functor Maybe where
  -- fmap :: (a -> b) -> Maybe a -> Maybe b
  fmap _ Nothing  = Nothing
  fmap f (Just x) = Just (f x)
  
  
instance Functor Tree where
  -- fmap :: (a -> b) -> Tree a -> Tree b
  fmap f (Leaf x)       = Leaf (f x)
  fmap f (Node lt x rt) = Node (fmap f lt) (f x) (fmap f rt)
```

As you can see, every time we extracted a more general expression we got a more powerful abstraction. But there are cases when `Functor` is not enough.

### The limits of `Functor`

Let's try applying a few functions to a `Functor`. Let's say the `Maybe` functor: 

In [32]:
:t (+1)             -- Int -> Int
:t (+1) <$> Just 3  -- Maybe Int
(+1) <$> Just 3

-----------------------------------------------------------------

-- Add (Just 3) and (Just 2)?
:t (+)             -- Int -> Int -> Int
:t (+) <$> Just 3  -- Maybe (Int -> Int)

-----------------------------------------------------------------

almostThere = (<$> Just 3) <$> ((+) <$> Just 2) 

:t  almostThere -- Maybe (Maybe Int)
almostThere

(+1) :: forall a. Num a => a -> a

(+1) <$> Just 3 :: forall b. Num b => Maybe b

Just 4

(+) :: forall a. Num a => a -> a -> a

(+) <$> Just 3 :: forall a. Num a => Maybe (a -> a)

almostThere :: forall b. Num b => Maybe (Maybe b)

Just (Just 5)

As you can see with the first example, we have no issue fmapping the a function +1 to Just 3. But what if we want to add two functors? If we check the result of applying the plus operator to Just 3, we get a function at the functor level. We are not prepared for this!

We can use fmap to take regular functions and apply them to functors. But we don't have a way to apply functions that already are at the functor level to functors. In the last example, we did some juggling to get almost what we needed. We managed to add the values but we ended up with duplicated functors.

The worst thing is that, this is a really common scenario. We may want to add two optional values that we retrieved from our database, maybe we want to apply a function to combine 3 trees into one, it's more common to have to work with multiple functors than with a single one. So, we need to solve this. 


One solution would be to create another versions of `fmap` that can handle a function of two arguments. For example, like this:

In [35]:
fmap2 :: (a -> b -> c) -> Maybe a -> Maybe b -> Maybe c
fmap2 f (Just a) (Just b) = Just (f a b)
fmap2 _ _ _               = Nothing



fmap2 (+) (Just 3) (Just 2)

Just 5

We take a binary function and two `Maybe` functors. If both values are `Just`, we apply the binary function, else, we return nothing because, if we are missing one or both values, we have no way of returning a valid result.

That would solve the problem. So in theory, we could create a type class called, for example:

```haskell
class Functor2 where
    fmap2 :: (a -> b -> c) -> f a -> f b -> f c
```

But that only works for binary functions. If we need to use a function that takes three, four, or five arguments, we would need to create:

```haskell
class Functor3 where
    fmap3 :: (a -> b -> c -> d) -> f a -> f b -> f c -> f d

class Functor4 where
    fmap4 :: (a -> b -> c -> d -> e) -> f a -> f b -> f c -> f d -> f e

class Functor5 where
    fmap5 :: (a -> b -> c -> d -> e -> g) -> f a -> f b -> f c -> f d -> f e -> f g
```

You know where we're going with this. This method is unsustainable. It doesn't matter how many type classes you create, there's always a usecase that requires more parameters. And, on top of that, we would have maybe dozens of type classes representing the same concept. 

But don't worry, thanks to currying, there's a better way!

# Function application at the `Functor` level 🥇

This is the key idea of the lesson. If you understand this section, the rest of the lecture will flow naturally.

Ok, so, we know that writing an `fmap` for every possible number of arguments is not feasable. But we don't have to. If we take a look at Haskell's function application:

```haskell
( ) :: (a -> b) -> a -> b
($) :: (a -> b) -> a -> b

result = (+1) 3 -- 4
--           ^
--           |
--    Function application
--            |
--            ⌄
result = (+1) $ 3
```

It takes a function that goes from `a` to `b` and a value of type `a` and returns a value of type `b`. It seems as if we could only apply a function of one argument to a value, but we know that in practice we can do this:

In [36]:
calculate :: Int -> Int -> Int -> Int
calculate a b c = a + b * c

calculate 3 6 4
--       ^ ^ ^
--       | | |
--    Function application

27

That is because all functions are curried, meaning they all take one argument and the `b` type returned by function application can be of any type, including a function. So, we can apply the `calculate` function to `3` to get a function that takes two parameters, then apply it to `6` to get a function that takes one parameter, and finally apply it to `4` to get an `Int`.

This is not news, we already knew all that from the first lessons of the course. What is new though, is that now we want to create an operator that works like function application, but for functors. That way, it will work for functions of any number of arguments.

So, if regular function application has this signature:

```haskell
( ) ::                 (a -> b) -> a -> b
```

How does the type signature of the same function would look if everything was a `Functor`? Well, you guessed it, like this:

```haskell
(<*>) :: Functor f => f (a -> b) -> f a -> f b
```

We call this the "star" or "ap" operator. We use an operator because, of course, it will be used mostly as an infix function. And the type reflects that the behavior is the same as regular function application but lifted to work at the functor level.

It's important not to confuse the signatures of this functor-level function application with fmap:

```haskell
(<$>) ::                (a -> b) -> f a -> f b
```

`fmap` takes a function that is not at the functor level and applies it to a functor. Once you applied the function the first time, the end result, be it the final value or a partially applied function, is now at the functor level. So we can not use `fmap` anymore.

Ok, so let's try defining it specifically for optional types. For `Maybe` values, this star operator would look like this: 

```haskell
(<*>) :: Maybe (a -> b) -> Maybe a -> Maybe b
Just f  <*> Just x  = Just (f x) 
_       <*> _       = Nothing 

```

We have to patternmatch to make sure that we actually have both the function and the value, apply the funciton to get a value of type `b`, and wrap it again with the Just constructor. If we don't have both, we can't return a valid result, so we return `Nothing`. 

This is really similar to the `fmap` definition. With the caveat that now we have to take into account that the function is optional too, not just the value.

Now that we have our definition, let's try it out!:

In [40]:
Just (+1) <*> (Just 3)

Just (+) <*> (Just 3) <*> (Just 2)

calculate :: Int -> Int -> Int -> Int
calculate a b c = a + b * c

Just calculate <*> (Just 3) <*> (Just 2) <*> (Just 4)

Just 4

Just 5

Just 11

Awesome, it seems that it works. But, I asured you that the star operator is function application at the functor level, and I compared its type with the function application type to build my case. It's time I prove it to you.

To prove that the star operator is function application at the functor level, I'm going to create a type that doesn't do anything. If the structure doesn't do anything, then the star operator should return the same result as regular function application, right? So let's do that.

How do you imagine a type that doesn't do anything looks like? It's actually pretty straight forward. We need a type that wraps another type and doesn't provide any functionality. Like this:

In [15]:
newtype Identity a = Identity { getIdentity :: a } deriving (Show, Eq)

As you can see, if we have a type with only one constructor that the only thing it does is to hold the value inside. It's virtually the same as the underlying type but with the extra anoyance of having to wrap and unwrap it.

We call this type the `Identity` type for the same reasons we call the identity values and the identity function like that. They preserve the identity of the values they interact with. They don't do anything basically.

Remember the newtype wrappers of the Semigroup and Monoid lessons? Those weren't identities because the behaviour we gave them when defining their Semigroup and Monoid instances actually did something. So now, we have to also implement the type class instances in a way that they have no effect.

Ok, so, let's implement the `Functor` instance for the `Identity` type:

In [16]:
instance Functor Identity where
  --fmap :: (a -> b) -> Identity a -> Identity b
  fmap f (Identity a) = Identity (f a)

To do that, we'll just pattern match to extract the internal value, apply the function and wrap it again. It's the same as the `Just` case of the `Maybe` type, with the difference that, because we only have one constructor, there's no option to fail, so we'll always be able to apply the function to the underlying value.

In [43]:
fmap (+1) (Identity 3) -- Identity 4

(*2) <$> Identity 3    -- Identity 6

'A' <$ Identity 3      -- Identity 'A'

fmap id (Identity False) == Identity False -- True

Identity {getIdentity = 4}

Identity {getIdentity = 6}

Identity {getIdentity = 'A'}

True

And as you can see, it works as expected and respects the `Functor` laws.

Now, we can create the star operator. 

The type was:

```haskell
(<*>) :: Functor f => f (a -> b) -> f a -> f b
```

So, we had a funciton at the functor level applied to a functor with values of type `a` to get the functor with values of type `b`. Because the `Identity` functor is basically just an anoying wrapper, for now at least, we know that we just have to unwrap and apply the funciton to the value of type `a`. So let's do that:

In [45]:
(<*>) :: Identity (a -> b) -> Identity a -> Identity b
(Identity f) <*> (Identity x) = Identity (f x)

infixl 4 <*>

Pretty straight forward, almost the same as `fmap`. Now comes the moment of truth. Let's see if the star operator is actually function application at the functor level:

In [46]:
(+1) 1
getIdentity $ Identity (+1) <*> Identity 1

(+) 1 2
getIdentity $ Identity (+) <*> Identity 1 <*> Identity 2

(\a b c -> a + b * c) 1 2 3
getIdentity $ Identity (\a b c -> a + b * c) <*> Identity 1 <*> Identity 2 <*> Identity 3

2

2

3

3

7

7

As you can see, we have three examples of applying the same funtion to a value or multiple values. Once without functor using the built-in function application, and once using the star operator within a `Functor` that doesn't do anything. And we get the same result in both cases.

We just proved that the star operator is a more generalized form of function application. It's more general because we get the same old function application if we apply it to the `Identity` functor, but we can also apply it to other functors.

Is this enough though? Let's talk about `pure`.

## Being `pure` 😇

We've proved that the star opearator is a generalized form of function application. Does that mean that we could handle all possible cases when working with Functors?

When working with Functors, there are three possible cases. You can have everything at the functor level:

In [11]:
Just (\a b c -> a + b * c) <*> Just 1 <*> Just 2 <*> Just 3

Just 7

No issue with that, it works using the star operator we just defined. 

Another case is to have the need to apply a function that is not a functor to a bunch of functors:

In [12]:
(\a b c -> a + b * c) <$> Just 1 <*> Just 2 <*> Just 3

Just 7

No issue either, that is what `fmap` is for.

But, we could also encounter a third scenario. One when one or more values aren't functors:

```haskell
--                              DB      Local    DB
Just (\a b c -> a + b * c) <*> Just 1 <*> 2 <*> Just 3
--                                        ^
--                                        |
--                              ❌  Not a Functor! ❌
```

This happens in dozens of scenarios. For example, if two of the three values com from the database and the other one is provided locally.

And we currently have no way to solve this. `fmap` lifts functions, not individual values. So, we need a new way to lift individual values. This function should take any value and return it embeded in a Functor. We could call this function `lift`, but for reasons we'll explain in the last section of this lesson, we'll call it `pure`:

```haskell
pure :: Functor f => a -> f a
```

Regardless of how it's called, what it does is to lift values to the functor level. 

We can define it trivially for the `Maybe` type like this:

In [25]:
pureMaybe :: a -> Maybe a
pureMaybe = Just

pureMaybe 'a'

pureMaybe 4

Just 'a'

Just 4

And having this funciton, we can solve the missing case like this:

In [24]:
Just (\a b c -> a + b * c) <*> Just 1 <*> pureMaybe 2 <*> Just 3

Just 7

Of course, `pureMaybe` is specialized to `Maybe` values. So, it's the same as wrapping the value in a `Just` constructor. We want a function that works for all `Functors`, so this will also be part of our new type class. 

And with this second behaviour, we can apply any function to any amount of values at the functor level. To prove it to you, I'm let's define fmap for all the funciton from zero to five arguments using these two new operators:

```haskell
    fmap0 :: a -> f a
    fmap0 = pure
    
    fmap1 :: (a -> b) -> f a -> f b
    fmap1 g fa = pure g <*> fa -- same as g <$> fa

    fmap2 :: (a -> b -> c) -> f a -> f b -> f c
    fmap2 g fa fb = pure g <*> fa <*> fb

    fmap3 :: (a -> b -> c -> d) -> f a -> f b -> f c -> f d
    fmap3 g fa fb fc = pure g <*> fa <*> fb <*> fc

    fmap4 :: (a -> b -> c -> d -> e) -> f a -> f b -> f c -> f d -> f e
    fmap4 g fa fb fc fd = pure g <*> fa <*> fb <*> fc <*> fd

    fmap5 :: (a -> b -> c -> d -> e -> g) -> f a -> f b -> f c -> f d -> f e -> f g
    fmap5 g fa fb fc fd fe = pure g <*> fa <*> fb <*> fc <*> fd <*> fe
```

I removed the `Functor` constraint to avoid cluttering the slide, but remember that all the `f`s are functors.

For `fmap0`, we have a function that takes zero arguments. That's a constant value. So, we need to embed a constant value into a functor. Easy, we use `pure` that fits perfectly.

For `fmap1`, we have a funciton that takes one argumment. We lift the function to the Functor level, and then use the star operator to apply it to an argument. 

For `fmap2`, we have a funciton that takes two argumments. We lift the function to the Functor level, and then use the star operator to apply it to both arguments.

And it's the same for `fmap3`, 4, and 5. Lift the function and use the star operator as many times as needed until the funciton is fully applied.

And, as I'm sure you might notice, `fmap1` is the same as our regular `fmap`. And `fmap2`, 3, 4, and 5, all start with the same `pure g <*> fa` pattern. So, we can simplify a bit by replacing the first appication of the star operator with a functor so we avoid having to lift the initial functor, and we get this:

```haskell
    fmap0 :: a -> f a
    fmap0 = pure
    
    fmap1 :: (a -> b) -> f a -> f b
    fmap1 g fa = g <$> fa

    fmap2 :: (a -> b -> c) -> f a -> f b -> f c
    fmap2 g fa fb = g <$> fa <*> fb

    fmap3 :: (a -> b -> c -> d) -> f a -> f b -> f c -> f d
    fmap3 g fa fb fc = g <$> fa <*> fb <*> fc

    fmap4 :: (a -> b -> c -> d -> e) -> f a -> f b -> f c -> f d -> f e
    fmap4 g fa fb fc fd = g <$> fa <*> fb <*> fc <*> fd

    fmap5 :: (a -> b -> c -> d -> e -> g) -> f a -> f b -> f c -> f d -> f e -> f g
    fmap5 g fa fb fc fd fe = g <$> fa <*> fb <*> fc <*> fd <*> fe
```

Awesome!! Looks like we only need the star operator and pure to handle all cases. I guess it's time to formalize our findings.

## The `Applicative` type class

Here's the `Applicative` type class: 

```haskell
class Functor f => Applicative f where
    pure  :: a -> f a
    (<*>) :: f (a -> b) -> f a -> f b
```

So, how can we express what an `Applicative` is in words? For starterd, could say something like this:

An `Applicative` is a functor with a function to embed expressions (`pure`), and an operator that works as function application at the functor level (`<*>`).

That's why they are called applicative functors, because they are functors with function application.

To define the `Applicative` type class, we just need these two new behaviours we just discovered. But before we formally define it, let's make it more concrete by defining a few instances so we have 

Let's start by defining the instance for the `Identity` type wrapper:

In [34]:
newtype Identity a = Identity { getValue :: a } deriving (Show, Eq)

instance Functor Identity where
  --fmap :: (a -> b) -> Identity a -> Identity b
  fmap f (Identity a) = Identity (f a)

Of course, because `Applicative` is a subclass of `Functor`, we have to define `Functor` first. It's pretty straight forward. We pattern match to extract the value wraped with the `Identity` constructor, apply the function to the underlying value, and wrap it again. Now to the new part:

In [37]:
instance Applicative Identity where
  -- pure :: a -> Identity a
  pure = Identity
  
  -- (<*>) :: Identity (a -> b) -> Identity a -> Identity b
  -- Identity f <*> Identity a = Identity (f a)
  Identity f <*> ia = fmap f ia -- same as definition above

To create an instance for the applicative type class, we need to define both `pure` and the star operator. We don't have much choice for `pure`. We need to lift a value to the `Identity` level and we have only one constructor with the same type as pure. So we just use that.

For the star operator, we have two choices. We can patternmatch to extract both the function and the value, apply the function to the value and wrap it again, or because we have `fmap` at our disposal, we can patternmatch to extract only the funciton and fmappit to the value. Which, if you look at the definition of the functior instance, it's the same as what we did in the first definition by hand, but we avoid repeating ourselves.

Now that the `Identity` type is an applicative functor, we can use the operators:

In [36]:
Identity (+1) <*> Identity 1

Identity (+) <*> Identity 1 <*> Identity 2

Identity (\a b c -> a + b * c) <*> Identity 1 <*> Identity 2 <*> Identity 3

Identity {getValue = 2}

Identity {getValue = 3}

Identity {getValue = 7}

That one was borring, so let's do a new one. Let's create an instance for the `Maybe` type. Same as before we start with the functor instance:

```haskell
data Maybe a = Nothing | Just a

-------------------------------------------------------------------------

instance Functor Maybe where
  -- fmap :: (a -> b) -> Maybe a -> Maybe b
  fmap _ Nothing  = Nothing
  fmap f (Just x) = Just (f x)

-------------------------------------------------------------------------

instance Applicative Maybe where
  -- pure :: a -> Maybe a
  -- pure = \x -> Nothing ❌
  pure = Just
  
  -- (<*>) :: Maybe (a -> b) -> Maybe a -> Maybe b
  Just f <*> ma = fmap f ma
  _      <*> _  = Nothing
```

We don't run this cell because these instances are alreay provided by the prelude.

So, we alredy talked about the `Functor` instance in the previous lesson, so we won't go there. Now, about the applicative instance...

We have two possible options that satisfy the types for `pure`. We could ignore the input and return `Nothing`, and we could use `Just` that fits perfectly fine by itself. Conceptually, it makes sense to .... . But, if that doesn't convince you, `Just` is the only options if we want to respect the laws of the type class!

## The `Applicative` laws

Remember that one of the characteristics of functors was that `fmap` din't change the structure itself. Here, the star operator has the same restriction.

Sadly, same as it happened with Semigroup, Monoid, and Functor, not everything is reflected in the type class definition. So, we have to complement it with a few laws. Here they are:

**Identity:**
```haskell
pure id <*> v = v
```

Applying a lifted `id` function to an applicative, should return the same applicative. This makes sense, right? Same as when we deal with `fmap`, the star operator should respect the structor of the Functor that is now an applicative functor. Let's see an example with the `Maybe` type:

In [58]:
(pure id <*> Just 3) == Just 3

--------------------------------------------

wrongPure = \x -> Nothing

(wrongPure id <*> Just 3) == Just 3

True

False

As you can see, the star operator should also respect the structure of the functor and only modify the values. If we don't do anything to the values, we should get our original applicatvie functor. That's why we have to use `Just` as `pure`. If we use a function like `wrongPure` that ignores the input and always returns `Nothing`, we are changing the the structure when applying a lifted id function.

Next, we have the Homomorphism law:

**Homomorphism:**
```haskell
pure f <*> pure x = pure (f x)
```

Applying a lifted function to a lifted value should be the same as applying the funciton to the value and then lift the result. This enforces that the  star operator has to behave the same as function application.

Let's see an example:

In [47]:
leftSide :: Maybe String
leftSide = pure show <*> pure 3

rightSide :: Maybe String
rightSide = pure (show 3)

leftSide == rightSide
leftSide

True

Just "3"

Amazing, works as expected!! We still have two more to go through. Next, we have the composition law:

**Composition:**
```haskell
(pure (.) <*> u <*> v) <*> w = u <*> (v <*> w) 
```

This one shows that the operator is associative, although it might not be clear at a glance, so here's an example:

In [50]:
leftSide  = (pure (.) <*> Just show <*> Just (*2)) <*> Just 3
rightSide =               Just show <*> (Just (*2) <*> Just 3)


leftSide == rightSide
leftSide

True

Just "6"

On the left side, we want to combine the `show` function with the  `(*2)` funtion first and then apply it to `3`. And we know that the way of combining functions is by composition. So, we lift function composition to the functor level and apply it to `Just show` and `Just (*2)`. Then, we apply the result of that comoposition to `Just 3`.

On the right side, we fist want to combine the last two and then combine the result with the first. In this case, we don't need to lift function composition because we can apply `Just (*2)` to `Just 3` directly. So we do that to get `Just 6` and then apply `Just show` to get the string representation of the number six.

As you can see, we get the same result on both sises.

Finally, we have the interchange law:

**Interchange:**
```haskell
f <*> pure x = pure (\f -> f x) <*> f
```

This one states that, we should get the same result if we flip the arguments when applying a function. We know this is true for regular function application. For example:

In [51]:
leftSide  = (+1) 3
rightSide = (\f -> f 3) (+1) 


leftSide == rightSide
leftSide

True

4

On the left side, we take the function `(+1)` and apply it to 3 to get 4.

On the right side, we take a function that takes another function as parameter and applies it to the number three and we apply this function to the `(+1)` funciton. This results in +1 being passed as parameter and being applied to 3.

As you can see, we have to make the extra step of adding a lambda function, but if we flip the arguments of function application, we stil get the same result. That property should also hold with our star operator:

In [52]:
leftSide  = Just (+1) <*> pure 3
rightSide = pure (\f -> f 3) <*> Just (+1) 


leftSide == rightSide
leftSide

True

Just 4

This is the same as we just saw but lifted at the appicative functor level.

Same as before, we have to make the extra step of adding a lambda function, but if we flip the arguments of the star operator, we stil get the same result.

And that's it! To hone in on the type class implementation, let's do one more before moving on. Let's create the Applicative instance for the `Either` type:

```haskell
instance Functor (Either e) where
  fmap f (Left e) = Left e
  fmap f (Right a) = Right (f a)

-----------------------------------------------------------

instance Applicative (Either e) where
  -- pure :: a -> Either a a
  pure = Right
  -- (<*>) :: Either e (a -> b) -> Either e a -> Either e b
  Left e <*> _ = Left e
  Right f <*> r = fmap f r
```

We use `Right` as the way to lift the value with `pure`. If we only follow the types, `Right` is our only choice. You can quickly check it by yourself. Also, it makes sense. Lifting anything to the `Left` constructor would be useless because it would make any function that uses it return `Left` regardless of the other values. On top of that, it also makes sense from the `Functor` perspective. `Left e` is part of the structure, and we want to lift a value, so we have to use a constructor that can hold a value, and the only one available is the `Right` constructor that can hold values of type `a`.

And for the star operator we have to take into account the cases that the function is both a `Left` and `Right`. If the function is a `Left`, that means we don't have a function, we have an error instead. So, we have no other option that propagating the error.

If the constructor is a `Right`, we do have the function. And in this case we could have both `Left` and `Right` as second argument, but we don't have to deal with that because we already defined `fmap` that  does it for us. So, we just `fmap` the funciton.

And that's it! If we try to use it, we see that the results are reasonable:

In [32]:
Right (+1) <*> pure 1

(+) <$> Right 1 <*> Right 2

(\a b c -> a + b * c) <$> pure 1 <*> Left 2 <*> pure 3

(\a b c -> a + b * c) <$> Right 1 <*> pure 2 <*> pure 3

Right 2

Right 3

Left 2

Right 7

But what we really want to check is if they follow the laws. So, let's do that. Starting with the identity law:

In [28]:
-- Identity: pure id <*> v = v


(pure id <*> Left 3) == Left 3

(pure id <*> Right 'a') == Right (id 'a')

True

True

Nice. Our implementation follows the identity law. Now let's check for the Homomorphism law:

In [53]:
-- Homomorphism: pure f <*> pure x = pure (f x)


leftSide :: Either e Int
leftSide = pure (+1) <*> pure 1

rightSide :: Either e Int
rightSide = pure ((+1) 1)


leftSide == rightSide

True

Awesome, 2/4 done. Let's do composition next:

In [30]:
-- Composition: pure (.) <*> u <*> v <*> w = u <*> (v <*> w)


leftSide :: Either String Int
leftSide = pure (.) <*> u <*> v <*> w
  where
    u = Right (+1)
    v = Right (*2)
    --v = Left "some error"
    w = Right 3

rightSide :: Either String Int
rightSide = u <*> (v <*> w)
  where
    u = Right (+1)
    v = Right (*2)
    -- v = Left "some error"
    w = Right 3


leftSide == rightSide

True

As you can see, this law holds as well. And, of course, we can change any of these three values to different ones, for example `Left "some error"`, and it will still hold.

And finally, we check for the interchange law:

In [54]:
-- Interchange: u <*> pure y = pure ($ y) <*> u


leftSide :: Either String Int
leftSide = u <*> pure 5
    where
        u = Right (+1)
        -- u = Left "some error"

rightSide :: Either String Int
rightSide = pure ($ 5) <*> u
    where
        u = Right (+1)
        -- u = Left "some error"


leftSide == rightSide

True

Amazing! Our implementation seems to follow all the `Applicative` laws. We now have the Identity, Maybe, and Either Applicatives under our belt. So, I think it's time to move on into programming with effects.

## 🎆 Programming with effects 🎆

Congratulations! You now know how to program with effects!! Oh, you didn't realize that? Well, you do! Here's why:

When we talked about `Maybe`, we said tha it gave use a way to represent values that might fail. We wanted to return a value, but we weren't sure if we were going to be able, so we used `Mabye`:

In [71]:
-- We used optional values to create safe versions of partial functions
divMaybe :: Int -> Int -> Maybe Int
divMaybe x 0 = Nothing
divMaybe x y = Just (x `div` y)

8 `divMaybe` 2
8 `divMaybe` 0

-- We used optional values to handle possible failed results
valueFromDB = Just 3 -- Fake Database call
valueFromDB

Just 4

Nothing

Just 3

In the first example, we see h

In the second example, ...

Now, imagine we need to apply the `calculate` function to a bunch of `Maybe` values:

In [72]:
calculate :: Int -> Int -> Int -> Int
calculate a b c = a + b * c

Before learning about `Functor` and `Applicative`, we could have defined a funciton that takes 3 `Maybe` values and applies `calculate` to them like this:

In [74]:
calculateMaybe ma mb mc = case ma of
    Nothing -> Nothing
    Just a -> case mb of
        Nothing -> Nothing
        Just b -> case mc of
            Nothing -> Nothing
            Just c -> Just (calculate a b c)


calculateExplicit (8 `divMaybe` 2) (pure 2) valueFromDB

Just 10

We create a new `calculateMaybe` function to manually handle all possible failure cases, and if all three of the values return a `Just`, we know that we have all three underlying numbers and we can apply the original `calculate` function to them. And, of course, we wrap the result in a `Just` constructor to get the correct type.

This has several disadvantages. One is that we have to create the new `calculateMaybe` function. Another is that `calculateMaybe` only works for `Maybe` values. And finally, we have to write a huge block pattern-matching with case expresions to handle the `Maybe` values three times.

But now that we have `Functor` and `Applicative`, we can do the same in one line:

In [75]:
calculate <$> (8 `divMaybe` 2) <*> (pure 2) <*> valueFromDB

Just 10

Not only works, but we also don't have to create a new function, we can replace the values we apply `calculate` to with any applicative functor and it will still work, and we don't have to handle the possible failure cases by hand. This is what we achieved. But there's more. 

We can also thnk of `Maybe` as a type that provides the simulated effect of exception handling. When we apply functions using the applicative operators, also called "apppicative style", we're actually doing the same as we do in `calculateMaybe` but in a more convenient way. That's why its understandable to be confused when I tell you we can think of it as an effect when you can clearly see in the body of `calculateMaybe` that it's actually pure code.

In Haskell we have a bit of a loose concept of effects. Sometimes, the same code can be thought as having an effect or not depending on who you ask. So, to make it as clear as possible, here are key clarifications that will save you a lot of time and headaches:

Haskell has both **real effects** and **simulated effects**.
- Real effects are inpure and unpredictable (e.g, `IO`)
- Simulated effects <u>look inpure</u> when you use them, but are <u>actually pure</u> under the hood (e.g., `Maybe`)

Simulated effects look inpure because there are computations that are hidden from us behind operators. So, from the point of view of the user of the operators, it looks like they are running side-effects, but it's actually pure code that they can not see.

Another worthy clarification is that:

Independently if they are real or simulated, **all effects** have an **associated type** and **<u>at least</u> an `Applicative` instance** (and usually a `Monad` instance as well).

You can think of the `Applicative` type class we cover in this lesson and the `Monad` type class that we'll cover in a future lesson, as the boundary between what is usually not considered an effect and what it is.

Let's see a few examples:

**Examples of effects in Haskell:**

- `Identity` provides no effect.
<br>

- `Maybe` provides the **simulated effect** of a computation that might fail.
<br>

- `Either` provides the **simulated effect** of a computation that might fail with an error message or some kind of context as to why we had an error.
<br>

- `IO` provides the **real effect** of interacting with the world (keyboard, screen, internet,...).

There are more effects we'll discuss in later lectures, but let's stick with these ones for now to avoid overwhelming ourselves.

And thinking on these terms, now it makes sense why we called `pure` to the function that we use to lift values to the applicative functor level.

It's because we're embeding a value without an effect, a pure value, into an effectful context.   

Once again, all we do with `pure` is to wrap the value with the correct expression, same as we have been doing during this lesson. That hasn't changed, it's the way we think about what we do that changed.

Now, with this point of view, let's finish this the same way we did with the `Functor` lesson, by exploring the extra functions we get with `Applicative` and how it's defined in the `base` library.

## Extra functions and `Applicative` as defined in `base`

An `Applicative` is a functor with operations to embed pure expressions (`pure`), and sequence computations and combine their results (`<*>`).

Finally, now that you know everthing about `Applicative`, let me introduce you to the functions available in `base` based on this type class:

d

---

You can think of if as we always had:

We apply a funciton to the `Maybe` functor and manage failure cases implicitly by the functions and operator definitions.

Or you can think of it as:

We perform the effect of evaluating all the possibly failing arguments and then apply the `calculate` computation to get the final result.

When we talk about an "effect" we talk about some extra calculation on top of calculating the value. In this case, the extra calculation is to evaluate each `Maybe` applicative functor that we want to apply the `calcualte` functor to, to know if we have a value or not. 

If you replace the definitions of the fmap and star operators:

In [68]:
calculateExplicit ma mb mc = case ma of
    Nothing -> Nothing
    Just a -> case mb of
        Nothing -> Nothing
        Just b -> case mc of
            Nothing -> Nothing
            Just c -> Just (calculate a b c)


calculateExplicit (8 `divMaybe` 2) (pure 2) valueFromDB

Just 10

You'll see that, under the hood, all what's happening is plain and borring error handling. So, the underlying mechanism is the same as always. 

Our interpretation of what's happening is the thing that changes. We attribute the concept of "optional failure effect" to the `Maybe` data type with its `Functor` and `Applicative` instances.

Notice that, we're not doing anything we couldn't do before learning about `Functor` and `Applicative`. We could have written the `calculateExplicit` function lessons ago. What we gained is being able to avoid explicitly handling every case by hand. We gained a framework, a set of type classes, functions, and operators that handle the dirty work for us so we can write terse and expressive code.

In the same vain than `Maybe`, when we deal with `Either` values like this:

In [69]:
valueFromDB = Right 3

divEither :: Int -> Int -> Either String Int
divEither x 0 = Left "division by zero"
divEither x y = Right (x `div` y)


calculate <$> (8 `divEither` 2) <*> pure 2 <*> valueFromDB

Right 10

Either provides a different effect. It's also the effect of an optional value, but on top of that it provides extra context in the case of failure.

And thinking on these terms, now it makes sense why we called `pure` to the function that we use to lift values to the applicative functor level.

It's because we're embeding a pure value, a value without the effect into the effectfull context. Once again, all we do is to wrap the value with the correct expression, same as we did during this lesson. It's the way we think about it that changed.

So, as an overview of the "effects" we worked with today, we have:

- `Identity` provides no effect.

- `Maybe` provides the effect of a computation that might fail.

- `Either` provides the effect of a computation that might fail with an error message or some kind of context as to why we had an error.